Load the Enron email dataset from Kaggle into a pandas dataframe. Kaggle has already expressed that this dataset has completely unique values and there are no missing values. Therefore no duplication removal is necessary (since HashMaps can only have unique keys) and no data imputation is necessary.

In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.linear_model import LinearRegression
from scipy.stats import uniform

In [2]:
# Clean the data
csv.field_size_limit(100000000)
emails_df = pd.read_csv('emails.csv', engine='python', on_bad_lines='skip')

print(emails_df.head())
print(emails_df.shape)
print(emails_df.dtypes)

# We really only care about the key data being clean here since the value data is irrelevant to training models. 
print(emails_df['file'].head(20))

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...
(460937, 2)
file       object
message    object
dtype: object
0        allen-p/_sent_mail/1.
1       allen-p/_sent_mail/10.
2      allen-p/_sent_mail/100.
3     allen-p/_sent_mail/1000.
4     allen-p/_sent_mail/1001.
5     allen-p/_sent_mail/1002.
6     allen-p/_sent_mail/1003.
7     allen-p/_sent_mail/1004.
8      allen-p/_sent_mail/101.
9      allen-p/_sent_mail/102.
10     allen-p/_sent_mail/103.
11     allen-p/_sent_mail/104.
12     allen-p/_sent_mail/105.
13     allen-p/_sent_mail/106.
14     allen-p/_sent_mail/107.

Implementation of a traditional HashMap, class basis is from "Geeks for Geeks" (https://www.geeksforgeeks.org/hash-map-in-python/). Utilizes separate chaining to resolve collisions. Size indicates the capacity of the hashmap, the 'keys' parameter is an array of keys, and the 'vals' parameter is an array of the values to insert with the keys. This class assumes that the keys and values arrays are of the same size. This class has functionality to insert values into the hashmap, get values out of the hashmap using the key, and return the number of total conflicts while inserting pairs.

In [3]:
class HashMap:
 
    # Create empty bin list of given size
    def __init__(self, size, keys, vals):
        self.size = size
        self.hash_table = self.create_bins()
        self.keys = keys
        self.vals = vals
        self.conflict_count = 0

        # Allows for initialization of the HashMap K/V pairs during its construction.
        for i in range(0, len(keys)):
            self.insert(keys[i], vals[i])
 
    def create_bins(self):
        return [[] for _ in range(self.size)]
 
    # Insert values into hash map
    def insert(self, key, val):
       
        # Get the index from the key
        # using hash function
        hashed_key = hash(key) % self.size
         
        # Get the bin corresponding to index
        bin = self.hash_table[hashed_key]
 
        found_key = False
        for index, pair in enumerate(bin):
            pair_key, pair_val = pair
             
            # check if the bin has same key as
            # the key to be inserted
            if pair_key == key:
                found_key = True
                break

            self.conflict_count += 1
 
        # If the bin has same key as the key to be inserted,
        # Update the key value
        # Otherwise append the new key-value pair to the bin
        if found_key:
            bin[index] = (key, val)
        else:
            bin.append((key, val))
 
    # Return searched value with specific key
    def get(self, key):
       
        # Get the index from the key using
        # hash function
        hashed_key = hash(key) % self.size
         
        # Get the bin corresponding to index
        bin = self.hash_table[hashed_key]
 
        found_key = False
        for index, pair in enumerate(bin):
            pair_key, pair_val = pair
             
            # check if the bucket has same key as 
            # the key being searched
            if pair_key == key:
                found_key = True
                break
 
        # If the bin has same key as the key being searched,
        # Return the value found
        # Otherwise indicate there was no record found
        if found_key:
            return pair_val
        else:
            return "No pair found"

    # Get the number of conflicts after insertions are completed.
    def get_conflicts(self):
        return self.conflict_count

Initialize a traditional hashmap to take in all the K/V pairs from the emails_df dataset with the 'file' column representing the keys and its corresponding 'message' column representing the values. The size of the HashMap is set to 100% the number of K/V pairs, so ideally with a perfect HashMap, there would be 0 collisions. 

In [4]:
keys = []
vals = []

for index, row in emails_df.iterrows():
    keys.append(row['file'])
    vals.append(row['message'])

hash_map = HashMap(len(keys), keys, vals)

Now check the number of collisions that occurred with the traditional HashMap.

In [68]:
print(f'Number of Hash Collisions: {hash_map.get_conflicts()}')
print(f'Average number of collisions per key: {hash_map.get_conflicts() / len(keys)}')

Number of Hash Collisions: 230729
Average number of collisions per key: 0.5005651531554204


This is a notable number of collisions. How can we modify this to result in less hash collisions? 
Introducing, the learned hash model. With the learned hash model, the index structure exhibited in the traditional hash map will be decomposed into a CDF model in which we can find the position of the desired key in the backing array.

Below is the RMINode class. This class takes in a subset of keys from the overall keys in the hashmap and creates a normalized CDF of the keys. A model is then trained on this CDF given the keys and will give the location of a key in the backing array. The CDF is used to utilize space most efficiently.

In [6]:
# Take in offset through binary string, left movements are a 0, right movements are a 1. Left movements are dictated on if the
# probability of a desired key with the specific CDF of 'this' RMINode <= 50, otherwise it is a right movement.

class RMINode:

    # Initialize an RMINode
    def __init__(self, keys, isNormal, maxMin):
        self.keys = keys
        self.model = self.generate_model(keys, isNormal, maxMin)

    # Generates a model using linear regression following either a normal distribution or uniform distribution.
    def generate_model(self, keys, isNormal, maxMin):
        feature_keys = np.array(keys).reshape(-1, 1)

        if isNormal:
            cdf = np.cumsum(keys) / np.sum(keys)
        else:
            cdf = uniform.cdf(keys, maxMin[1], maxMin[0])


        model = LinearRegression()
        model.fit(feature_keys, cdf)

        return model

    # Returns an array with two elements. The first element is the smaller half of the keys and the second element is the larger half of the keys.
    def split_key_set(self):
        return np.array_split(self.keys, 2)

    # Returns the predicted probability of a key
    def predict_key_probability(self, key):
        prediction = self.model.predict([[key]])[0]

        # unfortunately in application even with overfitting the data (which is beneficial for this model) there are still
        # instances of negative or >1 probabilities being returned which is not in the feasible range of predictions.
        if prediction > 1:
            return 1
        elif prediction < 0:
            return 0
        else:
            return prediction

Below is the RMI class which stands for "Recursive Model Index". An RMI models a full tree, meaning that every level of the tree is filled up to depth k. This has a few nice properties which can be utilized for the learned hash function. Firstly a full tree can be modeled through an array rather than an actual tree structure which helps keep things efficient. Secondly, the number of nodes in a full tree can always be calculated through 2^(k+1) - 1 where k is the depth of the tree. It can be observed that this is exponential. As such, more depth to the full tree allows for greater precision but consumes much more cost computationally. k should be kept relatively low to keep the hash lookups in constant, O(1), time. Another useful property of full trees is that the number of nodes on the deepest (leaf) level is 2^k. 

RMI uses the full tree properties to instaniate a backing array of size 2^(k+1) - 1 and fills it with RMINodes (see documentation above). These RMINodes each contain a subset of the sorted keys and predict a key's position in its CDF given the key value. Once the leaf level is reached, RMI accounts for the offset from left-to-right of the sorted length(keys)/(2^k) partitions. Using this, it narrows down the position of a key in the backing array and returns it as the index to search.

In [34]:
class RMI:

    def __init__(self, keys, k, isNormal):
        self.keys = np.sort(np.array(keys))
        self.k = k
        self.isNormal = isNormal
        self.hash_table = self.create_hash_table()

        # This is the leaf level
        self.smallest_partition_size = len(keys) / (2 ** k)
        self.first_leaf_index = len(self.hash_table) - (2 << (self.k-1))

    # Using the depth k given, create a full tree of RMINodes
    def create_hash_table(self):
        hash_table = [None] * (2 ** (self.k + 1) - 1)
        self.fill_hash_table_recursive(hash_table, 0, self.keys)
        return hash_table

    # Recursively adds RMINodes to the full tree, splitting the dataset in half for each child node. 
    def fill_hash_table_recursive(self, hash_table, index, subkeys):

        # Base case
        if index >= len(hash_table):
            return

        hash_table[index] = RMINode(subkeys, self.isNormal, (subkeys[len(subkeys)-1], subkeys[0]))

        # Fill the left and right children of this node, in a full tree (which the RMI is) the left child can be accessed from 2 * parent_index + 1,
        # and the right child can be accessed from 2 * parent_index + 2

        subset_keys = hash_table[index].split_key_set()
        self.fill_hash_table_recursive(hash_table, 2 * index + 1, subset_keys[0])
        self.fill_hash_table_recursive(hash_table, 2 * index + 2, subset_keys[1])

    # Starting at the root RMINode, check the key's probability from the CDF and if it's less than or equal to 50%, go to the left child node,
    # else go to the right child node and keep track of changes through a binary string such that left movements are a '0' and right movements
    # are a '1'. 
    def get_hash_index(self, key):
        cur_index = 0
        key_probability = 0
        key_subset_offset = 0

        # Keep narrowing down the search until the leaf nodes are reached.
        while cur_index < self.first_leaf_index:
            key_probability = self.hash_table[cur_index].predict_key_probability(key)

            if key_probability <= 0.5:
                # Bitshift to the left one
                key_subset_offset = key_subset_offset << 1
                cur_index = 2 * cur_index + 1
            else:
                # Bitshift to the left one and bitwise OR a 1
                key_subset_offset = (key_subset_offset << 1) | 1
                cur_index = 2 * cur_index + 2

        return round((key_probability + key_subset_offset) * self.smallest_partition_size)

Now the RMI class will be used as the basis for the Learned Hash Map. The Learned Hash Map has the same functionality as the traditional hashmap but some additional parameters must be specified. The size parameter specifies the capacity of the hashmap, the keys parameter is an array of keys, and the values parameter is an array of values of the same size. Then, a value 'k' must be given which is the depth of of the internal RMI. Details on the implications of depth 'k' are given in the RMI class documentation. One important note is that your value of k will have 2^k leaf nodes in the RMI. As such you must have at least as many values to insert as the number of leaf nodes plus one. Secondly the value for k must be at least 2 to get actual benefit out of it. Hence, technically you can not make a hashmap smaller than size 5, but this class in general considers large datasets. Another parameter is the 'isNormal' which specifies whether the data should be approximated to a normal distribution (if true) or a uniform distribution (if false). Finally, the 'isText' parameter specifies whether text data is being used for the keys or not. Random variables map an item in the sample space to a number in the real number range. We couldn't find a good way to do this while still being O(1) time, so we had to use an internal dictionary to store which string key maps to which encoding. This is only used if isText is true, if the keys are numerical then this should be false. The hashmap currently has the functionality to insert values, get values from it, and get the total number of conflicts with insertions. 

In [62]:
class LearnedHashMap:

    # Create empty bin list of given size
    def __init__(self, size, keys, vals, k, isNormal, isText):
        if k < 2:
            raise Exception('You must at least have a depth of 2!')
        if len(keys) < ((2 ** k) + 1):
            raise Exception('You must have at least as many total values plus one to have one element per leaf of the RMI! ' + 
                            f'You must have at least {((2 ** k) + 1)} values to use this value of k!')
        
        self.size = size
        self.hash_table = self.create_bins()
        self.keys = keys
        self.vals = vals
        self.conflict_count = 0
        self.isText = isText
        self.keyToValue = {}
        self.availableEncoder = 0

        # Initialize the RMI with the keyset. If there is text data then assign unique label encoders for each key.
        if self.isText:
            encoders = []
            for i in range (0, len(keys)):
                encoders.append(i)
            self.RMI = RMI(encoders, k, isNormal)
        else:
            self.RMI = RMI(keys, k, isNormal)

        # Allows for initialization of the HashMap K/V pairs during its construction.
        for i in range(0, len(keys)):
            self.insert(keys[i], vals[i])

    def create_bins(self):
        return [[] for _ in range(self.size)]

    # Insert values into hash map
    def insert(self, key, val):

        if self.isText:
            if self.keyToValue.get(key) == None:
                self.keyToValue[key] = self.availableEncoder
                key = self.availableEncoder
                self.availableEncoder += 1
            else:
                key = self.keyToValue[key]

        # Get the index from the key
        # using hash function
        hashed_key = self.RMI.get_hash_index(key)

        if hashed_key > len(self.keys) - 1:
            hashed_key = len(self.keys) - 1

        # Get the bin corresponding to index
        bin = self.hash_table[hashed_key]

        found_key = False
        for index, pair in enumerate(bin):
            pair_key, pair_val = pair

            # check if the bin has same key as
            # the key to be inserted
            if pair_key == key:
                found_key = True
                break

            self.conflict_count += 1

        # If the bin has same key as the key to be inserted,
        # Update the key value
        # Otherwise append the new key-value pair to the bin
        if found_key:
            bin[index] = (key, val)
        else:
            bin.append((key, val))

    # Return searched value with specific key
    def get(self, key):

        if self.isText:
            key = self.keyToValue.get(key)
            if key == None:
                return "No value found"

        # Get the index from the key using
        # hash function
        hashed_key = self.RMI.get_hash_index(key)

        if hashed_key > len(self.keys) - 1:
            hashed_key = len(self.keys) - 1

        # Get the bin corresponding to index
        bin = self.hash_table[hashed_key]

        found_key = False
        for index, pair in enumerate(bin):
            pair_key, pair_val = pair

            # check if the bucket has same key as
            # the key being searched
            if pair_key == key:
                found_key = True
                break

        # If the bin has same key as the key being searched,
        # Return the value found
        # Otherwise indicate there was no record found
        if found_key:
            return pair_val
        else:
            return "No value found"

    # Get the number of conflicts after insertions are completed.
    def get_conflicts(self):
        return self.conflict_count

Now, using the learned hash function 

In [63]:
# Load in the enron email dataset with an RMI depth of 3. Since these keys are all unique a uniform distribution is a better approximation
# than a normal distribution. The isText parameter is true since the keys are strings.
hash_table_l = LearnedHashMap(len(keys), keys, vals, 3, False, True)

806618


In [64]:
print(f'Number of Hash Collisions: {hash_table_l.get_conflicts()}')
print(f'Average number of collisions per key: {hash_table_l.get_conflicts() / len(keys)}')

Number of Hash Collisions: 806618
Average number of collisions per key: 1.7499528135081366


With the learned hashmap, we surprisingly see more more conflicts on average "1.7499528135081366" as compared to the traditional hashmap with
0.5005651531554204. This is good to know but may not be a definitive factor. Before we go on, as seen below, both hash maps do work properly and return the same value when the same key is inputted.

In [67]:
key = keys[50]
val = vals[50]

print(key)

print(hash_map.get(key))
print(hash_table_l.get(key))

allen-p/_sent_mail/14.
Message-ID: <27936946.1075855378542.JavaMail.evans@thyme>
Date: Wed, 2 May 2001 10:27:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tori.kuykendall@enron.com
Subject: Re: 2- SURVEY - PHILLIP ALLEN
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tori Kuykendall <Tori Kuykendall/HOU/ECT@ECT>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst


---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/02/2001 05:26 AM ---------------------------


Ina Rangel
05/01/2001 12:24 PM
To:	Phillip K Allen/HOU/ECT@ECT
cc:	 
Subject:	Re: 2- SURVEY - PHILLIP ALLEN   




   
-
Full Name:        Phillip Allen

Login ID:  	pallen

Extension:  3-7041

Office Location:  EB3210C

What type of computer do you have?  (Desktop,  Laptop,  Both)  Both

Do you have a PDA?  If yes, what type do you have:   (None, IPAQ, 

Let's try a couple more tests. How about we try a 'k' depth of 2 and one of 4? Theoretically, the more depth of k you have, the more precision the RMI structure has with its models.

In [69]:
# Load in the enron email dataset with an RMI depth of 2. Since these keys are all unique a uniform distribution is a better approximation
# than a normal distribution. The isText parameter is true since the keys are strings.
hash_table_l_2 = LearnedHashMap(len(keys), keys, vals, 2, False, True)

In [70]:
print(f'Number of Hash Collisions: {hash_table_l_2.get_conflicts()}')
print(f'Average number of collisions per key: {hash_table_l_2.get_conflicts() / len(keys)}')

Number of Hash Collisions: 460935
Average number of collisions per key: 0.9999956610122425


Surprisingly, we see about half the number of collisions as with a 'k' value of 3. This could be due to the fact that uniform distribution CDFs are a perfectly linear line, as such more models might actually be leading to more uncertainty which increases hash collisions unnecessarily since there isn't a lot of variation in the CDF. Regardless, let's try with a 'k' value of 4 and see if we may be correct.

In [71]:
# Load in the enron email dataset with an RMI depth of 4. Since these keys are all unique a uniform distribution is a better approximation
# than a normal distribution. The isText parameter is true since the keys are strings.
hash_table_l_4 = LearnedHashMap(len(keys), keys, vals, 4, False, True)

In [72]:
print(f'Number of Hash Collisions: {hash_table_l_4.get_conflicts()}')
print(f'Average number of collisions per key: {hash_table_l_4.get_conflicts() / len(keys)}')

Number of Hash Collisions: 1497973
Average number of collisions per key: 3.2498432540672586


It appears that our earlier hypothesis may be correct with regards to the uniform CDF. An increased value of 'k' may just be leading to further uncertainty rather than flattening out variation in the CDF (since there is none). Now let's try some randomly, synthetically produced data from a normal distribution, and compare to the traditional hashmap. 

In [87]:
# This generates 100,000 values from a normal distribution with a mean of 5,000 and a standard deviation of 10.
keys_norm = np.random.normal(5000, 10, 100000)

hash_map_norm = HashMap(len(keys_norm), keys_norm, keys_norm)

In [88]:
print(f'Number of Hash Collisions: {hash_map_norm.get_conflicts()}')
print(f'Average number of collisions per key: {hash_map_norm.get_conflicts() / len(keys_norm)}')

Number of Hash Collisions: 52463
Average number of collisions per key: 0.52463


In [89]:
# The learned hashmap will use the keys_norm as both the key and value with a RMI depth of 3, 'isNormal' set to true since we are
# approximating a normal distribution, and 'isText' set to false since the keys are numerical data.
hash_table_lnorm = LearnedHashMap(len(keys_norm), keys_norm, keys_norm, 3, True, False)

In [90]:
print(f'Number of Hash Collisions: {hash_table_lnorm.get_conflicts()}')
print(f'Average number of collisions per key: {hash_table_lnorm.get_conflicts() / len(keys_norm)}')

Number of Hash Collisions: 20034373
Average number of collisions per key: 200.34373


This was actually a huge difference. Although 200 is still small enough to not be considered proportional to the input size of 100,000, considering that the average for the traditional hashmap was 0.5 collisions per key, there is a huge difference in performance. Our hypothesis after running this and doing many debugging tests is that the models in the RMINodes may be having a difficult time with the normal approximation method. There are other methods to try, but we ran into a lot of deadends and eventually just ran out of time.

Finally, using the best learned hashmap with a 'k' of 2 containing the enron email dataset, we will compare its runtime to lookup 100,000 keys in random order to the same on the traditional hashmap.

In [92]:
import time
import random

In [98]:
# randomly permute the order of items in the keys array, this will allow for random lookups.
random.shuffle(keys)

In [111]:
start_time = time.time()

for i in range(0, 100000):
    hash_map.get(keys[i])

mid_time = time.time()

for i in range(0, 10000):
    pass
    # Gets the overhead of the loop

stop_time = time.time()

print(f'Total time to randomly lookup all keys with the traditional hashmap: {(mid_time - start_time) - (stop_time - mid_time)}')

Total time to randomly lookup all keys with the traditional hashmap: 0.13964319229125977


In [112]:
start_time = time.time()

for i in range(0, 100000):
    hash_table_l_2.get(keys[i])

mid_time = time.time()

for i in range(0, 10):
    pass
    # Gets the overhead of the loop

stop_time = time.time()

print(f'Total time to randomly lookup all keys with the learned hashmap with k = 2: {(mid_time - start_time) - (stop_time - mid_time)}')

Total time to randomly lookup all keys with the learned hashmap with k = 2: 5.924741268157959


Analyzing this runtime, it does seem that the traditional hashmap has better data distribution than our learned hashmap does. Although it isn't perfect, this was definitely a great learning experience and has lots of room for improvement in the future with different models and neural networks. 